# Introduction 

This note evaluate smolLM-135M instruct model with llama.cpp

# Env and tools prepare

In [1]:
!git clone https://github.com/ggerganov/llama.cpp
%cd llama.cpp
# !export GGML_CUDA=1 
!cp -r /usr/local/cuda-12.3/targets /usr/local/nvidia/ 
!make GGML_CUDA=1 CUDA_PATH=/usr/local/nvidia  > make.log 2>&1

Cloning into 'llama.cpp'...
remote: Enumerating objects: 34776, done.
remote: Counting objects: 100% (6485/6485), done.
remote: Compressing objects: 100% (354/354), done.
remote: Total 34776 (delta 6297), reused 6199 (delta 6130), pack-reused 28291 (from 1)
Receiving objects: 100% (34776/34776), 56.54 MiB | 22.92 MiB/s, done.
Resolving deltas: 100% (25233/25233), done.
/kaggle/working/llama.cpp


In [2]:
!tail make.log
!pip install --force-reinstall -r requirements.txt > pip_install.log 2>&1
!tail pip_install.log

c++ -std=c++11 -fPIC -O3 -g -Wall -Wextra -Wpedantic -Wcast-qual -Wno-unused-function -Wmissing-declarations -Wmissing-noreturn -pthread -fopenmp  -march=native -mtune=native -Wno-array-bounds -Wno-format-truncation -Wextra-semi -Iggml/include -Iggml/src -Iinclude -Isrc -Icommon -D_XOPEN_SOURCE=600 -D_GNU_SOURCE -DNDEBUG -DGGML_USE_OPENMP -DGGML_USE_LLAMAFILE -DGGML_USE_CUDA -DGGML_CUDA_USE_GRAPHS -I/usr/local/nvidia/include -I/usr/local/nvidia/targets/x86_64-linux/include  -c examples/cvector-generator/cvector-generator.cpp -o examples/cvector-generator/cvector-generator.o
c++ -std=c++11 -fPIC -O3 -g -Wall -Wextra -Wpedantic -Wcast-qual -Wno-unused-function -Wmissing-declarations -Wmissing-noreturn -pthread -fopenmp  -march=native -mtune=native -Wno-array-bounds -Wno-format-truncation -Wextra-semi -Iggml/include -Iggml/src -Iinclude -Isrc -Icommon -D_XOPEN_SOURCE=600 -D_GNU_SOURCE -DNDEBUG -DGGML_USE_OPENMP -DGGML_USE_LLAMAFILE -DGGML_USE_CUDA -DGGML_CUDA_USE_GRAPHS -I/usr/local/nvidi

# Dowload wiki text



In [ ]:
!wget https://object.pouta.csc.fi/OPUS-Wikipedia/v1.0/mono/en.txt.gz
!gunzip en.txt.gz
!head -n 10000 en.txt > en-h10000.txt
!sh /kaggle/working/llama.cpp/scripts/get-wikitext-2.sh

# Run perplexity

Perplexity is the main evaluation metric for large language models (LLMs). It measures how well the model predicts a given sequence of tokens.
Formally, the perplexity of LLMs is the exponentiated average negative log-likelihood,  which means **perplexity smaller is better** 

Perplexity benchmark shows how accurate the model is 

In [12]:

!/kaggle/working/llama.cpp/llama-perplexity -m  /kaggle/input/smollm-135m-instruct-gguf/gguf/q4_k_m/1/SmolLM-135M-Instruct-Q4_K_M-v2.gguf -f /kaggle/working/llama.cpp/wikitext-2-raw/wiki.test.raw --chunks 256 > ppl_Q4_V2.log 2>&1
!/kaggle/working/llama.cpp/llama-perplexity -m  /kaggle/input/smollm-135m-instruct-gguf/gguf/q4_k_m/1/SmolLM-135M-Instruct-Q4_K_M.gguf -f /kaggle/working/llama.cpp/wikitext-2-raw/wiki.test.raw --chunks 256 > ppl_Q4.log 2>&1
!/kaggle/working/llama.cpp/llama-perplexity -m  /kaggle/input/smollm-135m-instruct-gguf/gguf/fp16/1/SmolLM-135M-Instruc-fp16.gguf -f /kaggle/working/llama.cpp/wikitext-2-raw/wiki.test.raw --chunks 256 > pip_fp16.log 2>&1


In [13]:
!tail -6 ppl_Q4_V2.log
!tail -6 ppl_Q4.log
!tail -6 pip_fp16.log

Final estimate: PPL = 23.2651 +/- 0.28000

llama_perf_context_print:        load time =    1218.49 ms
llama_perf_context_print: prompt eval time =   28713.83 ms / 131072 tokens (    0.22 ms per token,  4564.77 tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =   49529.15 ms / 131073 tokens
Final estimate: PPL = 23.2651 +/- 0.28000

llama_perf_context_print:        load time =    1120.58 ms
llama_perf_context_print: prompt eval time =   28658.76 ms / 131072 tokens (    0.22 ms per token,  4573.54 tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =   49226.36 ms / 131073 tokens
Final estimate: PPL = 21.2879 +/- 0.25264

llama_perf_context_print:        load time =    2234.91 ms
llama_perf_context_print: prompt eval 

# Benchmarking the Inference Throughput and Memory Consumption 

Llama-bench shows performance and resource consumption. 
There are three types of test
- Prompt processing (pp): processing a prompt in batches (-p)
- Text generation (tg): generating a sequence of tokens (-n)
- Prompt processing + text generation (pg): processing a prompt followed by generating a sequence of tokens (-pg)

In the following test, we run text generation and prompt processing


In [23]:
!/kaggle/working/llama.cpp/llama-bench -m /kaggle/input/smollm-135m-instruct-gguf/gguf/q4_k_m/1/SmolLM-135M-Instruct-Q4_K_M-v2.gguf -m /kaggle/input/smollm-135m-instruct-gguf/gguf/q4_k_m/1/SmolLM-135M-Instruct-Q4_K_M.gguf -m /kaggle/input/smollm-135m-instruct-gguf/gguf/fp16/1/SmolLM-135M-Instruc-fp16.gguf -mg 1 -ngl 10,20,30,35 -n 128,256,512 

ggml_cuda_init: GGML_CUDA_FORCE_MMQ:    no
ggml_cuda_init: GGML_CUDA_FORCE_CUBLAS: no
ggml_cuda_init: found 1 CUDA devices:
  Device 0: Tesla P100-PCIE-16GB, compute capability 6.0, VMM: yes
| model                          |       size |     params | backend    | ngl |   main_gpu |          test |                  t/s |
| ------------------------------ | ---------: | ---------: | ---------- | --: | ---------: | ------------: | -------------------: |
| llama ?B Q4_K - Medium         | 127.55 MiB |   162.83 M | CUDA       |  10 |          1 |         pp512 |     7828.97 ± 179.35 |
| llama ?B Q4_K - Medium         | 127.55 MiB |   162.83 M | CUDA       |  10 |          1 |         tg128 |         81.26 ± 1.06 |
| llama ?B Q4_K - Medium         | 127.55 MiB |   162.83 M | CUDA       |  10 |          1 |         tg256 |         78.49 ± 3.45 |
| llama ?B Q4_K - Medium         | 127.55 MiB |   162.83 M | CUDA       |  10 |          1 |         tg512 |         77.29 ± 1.67 |
| llama ?B Q4_K -

# Package llama_cpp

In [32]:
!pwd
!cd /kaggle/working/
!tar -czf /kaggle/working/llama.tar.gz /kaggle/working/llama.cpp

/kaggle/working/llama.cpp
tar: Removing leading `/' from member names


In [34]:
!ls /kaggle/working/

llama.cpp  llama.tar.gz
